In [1]:

%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:

import pandas as pd
import numpy as np
import imutils
import glob
import cv2
import shutil
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
## Loading Haar Cascade
## Taken from https://github.com/opencv/opencv/tree/master/data/haarcascades
# We are going to use OpenCV Haar Cascades to find the location of the face in the image and crop to only keep the facial parts of the image.
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [29]:
def face_extractor(origin, destination, fc):
    ## Importing image using open cv
    img = cv2.imread(origin,1)

    ## Resizing to constant width
    img = imutils.resize(img, width=200)
    
    ## Finding actual size of image
    H,W,_ = img.shape
    
    ## Converting BGR to RGB
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    ## Detecting faces on the image
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

    img = cv2.imread(destination)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = fc.detectMultiScale(gray, 1.3, 5)
    
    ## If only one face is found
    if len(face_coord) == 1:
        X, Y, w, h = face_coord[0]
    
    ## If no face found --> SKIP
    elif len(face_coord)==0:
        return None
    
    ## If multiple faces are found take the one with largest area
    else:
        max_val = 0
        max_idx = 0
        for idx in range(len(face_coord)):
            _, _, w_i, h_i = face_coord[idx]
            if w_i*h_i > max_val:
                max_idx = idx
                max_val = w_i*h_i
            else:
                pass
            
            X, Y, w, h = face_coord[max_idx]
    
    ## Crop and export the image
    img_cp = img[
            max(0,Y - int(0.35*h)): min(Y + int(1.35*h), H),
            max(0,X - int(w*0.35)): min(X + int(1.35*w), W)
        ].copy()
    
    cv2.imwrite(destination, img_cp)

In [18]:
# get the frame by frame of the video
# using this because cv2.imshow is having so much error
vidcap = cv2.VideoCapture('TestVideo.mp4')
def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
        cv2.imwrite("PS/image"+str(count)+".png", image)     # save frame as JPG file
    return hasFrames
sec = 0
frameRate = 1 #//it will capture image in each 0.5 second
count=1
success = getFrame(sec)
while success:
    count = count + 5
    sec = sec + frameRate
    sec = round(sec, 2)
    success = getFrame(sec)

In [19]:
## Defining destination path
path = '/content/PS'

## Finding all the images in the folder
item_list = glob.glob('/content/PS/*.png')
print(len(item_list))

23


In [14]:
%%capture
pip install tqdm

In [30]:
import tqdm
for org in tqdm.notebook.tqdm(item_list):
    face_extractor(origin = org, destination = path+org.split('/')[-1], fc=face_cascade)

error: ignored